In [1]:
import collections

import numpy as np

import util
import svm

In [2]:
train_messages, train_labels = util.load_spam_dataset('spam_train.tsv')
val_messages, val_labels = util.load_spam_dataset('spam_val.tsv')

In [3]:
train_messages

['THANX 4 PUTTIN DA FONE DOWN ON ME!!',
 "So how are you really. What are you up to. How's the masters. And so on.",
 "Joy's father is John. Then John is the NAME of Joy's father. Mandan",
 'Almost there, see u in a sec',
 'Yes baby! We can study all the positions of the kama sutra ;)',
 'Great! So what attracts you to the brothas?',
 'So dont use hook up any how',
 'It shall be fine. I have avalarr now. Will hollalater',
 'watever reLation u built up in dis world only thing which remains atlast iz lonlines with lotz n lot memories! feeling..',
 'This is my number by vivek..',
 'Hi good mornin.. Thanku wish u d same..',
 'Idea will soon get converted to live:)',
 'HEY GIRL. HOW R U? HOPE U R WELL ME AN DEL R BAK! AGAIN LONG TIME NO C! GIVE ME A CALL SUM TIME FROM LUCYxx',
 'Tell your friends what you plan to do on Valentines day @ &lt;URL&gt;',
 'YOUR CHANCE TO BE ON A REALITY FANTASY SHOW call now = 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national =

In [5]:
test_string = train_messages[0]

In [3]:

def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For simplicity, you should split on whitespace, not
    punctuation or any other character. For normalization, you should convert
    everything to lowercase.  Please do not consider the empty string (" ") to be a word.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """

    # *** START CODE HERE ***
    
    message = message.lower()

    return message.split(sep=" ") 
    # *** END CODE HERE ***


def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message.

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """

    # *** START CODE HERE ***
    messages_str =  " ".join(messages)
    unique_words = set(get_words(messages_str))
    i=0
    freq_dict = dict.fromkeys(unique_words,0)
    for message in messages:
        for word in set(get_words(message)):
            freq_dict[word] += 1
            
    word_dict = dict()
    i=0
    for word in freq_dict:
        if freq_dict[word]>=5:
            i+=1
            word_dict[word]=i
    return word_dict
    # *** END CODE HERE ***

In [4]:
word_dict = create_dictionary(train_messages)

In [5]:
def transform_text(messages, word_dictionary):
    msg_token = np.zeros((len(messages),len(word_dictionary)))
    
    for row, msg in enumerate(messages):
        for word in get_words(msg):
            try:
                msg_token[row][word_dictionary[word]] += 1
            except:
                pass
    
    return msg_token
    
    

transform_text(train_messages, word_dict)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [22]:
word_dict

{'': 1,
 'caller': 2,
 'down': 3,
 'photo': 4,
 'howz': 5,
 'ya,': 6,
 'ignore': 7,
 'treat': 8,
 'was': 9,
 'voda': 10,
 'update': 11,
 'added': 12,
 'pick': 13,
 'statement': 14,
 'really': 15,
 'after': 16,
 'support': 17,
 'attempt': 18,
 'wonderful': 19,
 'th': 20,
 "what's": 21,
 'your': 22,
 'top': 23,
 'find': 24,
 'music': 25,
 'lovely': 26,
 'needs': 27,
 'sunny': 28,
 'ever': 29,
 'project': 30,
 'mind': 31,
 'giving': 32,
 'sir.': 33,
 'here.': 34,
 '*9': 35,
 'weekend': 36,
 'decide': 37,
 'amazing': 38,
 'wishing': 39,
 'expires': 40,
 'email': 41,
 'home.': 42,
 ':-)': 43,
 'tomorrow?': 44,
 'much': 45,
 'sexy': 46,
 '5': 47,
 'away': 48,
 'lemme': 49,
 'card': 50,
 'looking': 51,
 'again': 52,
 'yar': 53,
 'speak': 54,
 ':)': 55,
 'air': 56,
 'checking': 57,
 '86688': 58,
 'quoting': 59,
 'services': 60,
 'drop': 61,
 'dont': 62,
 'carlos': 63,
 'wish.': 64,
 'pain.': 65,
 'room.': 66,
 'yesterday.': 67,
 'dream': 68,
 'b4': 69,
 'aight,': 70,
 'you': 71,
 '£10,000': 72

In [25]:
list(word_dict)[[]

TypeError: list indices must be integers or slices, not list

In [5]:
len(word_dict)

1717

In [6]:
len(train_messages)

4459

In [13]:
def fit_naive_bayes_model(matrix, labels):
    """Fit a naive bayes model.

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """

    # *** START CODE HERE ***
    phi = dict()
    recs, words_count = matrix.shape

    matrix1 = matrix[labels==1,:]
    matrix0 = matrix[labels==0,:]

    phi['1'] = (1+ matrix1.sum(axis=0)) / (np.sum(matrix1.sum(axis=1)) + words_count)
    phi['0'] = (1+ matrix0.sum(axis=0)) / (np.sum(matrix0.sum(axis=1)) + words_count)
    phi['all'] = matrix1.shape[0]/(matrix1.shape[0]+matrix0.shape[0])
    return phi

In [6]:
word_matrix = transform_text(train_messages, word_dict)

In [17]:
word_matrix[train_labels==1,:].sum(axis=0).shape

(1717,)

In [18]:
word_matrix[train_labels==1,:].sum(axis=1).shape

(611,)

In [19]:
word_matrix[train_labels==1,:].shape

(611, 1717)

In [16]:
nb_model = fit_naive_bayes_model(word_matrix, train_labels)

In [17]:
def predict_from_naive_bayes_model(model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: A numpy array containg the predictions from the model
    """
    # *** START CODE HERE ***
    predictions = np.zeros(matrix.shape[0])

    log_phi1 = np.sum(np.log(model['1'])*matrix, axis=1)
    log_phi0 = np.sum(np.log(model['0'])*matrix, axis=1)
    phi = model['all']

    probabilty = 1 / (1 + np.exp(log_phi0 + np.log(1-phi) - log_phi1 - np.log(phi)))

    return np.round(probability)

predict_from_naive_bayes_model(nb_model, word_matrix)

TypeError: 'axis' is an invalid keyword to ufunc 'log'

In [31]:
def compute_best_svm_radius(train_matrix, train_labels, val_matrix, val_labels, radius_to_consider):
    """Compute the optimal SVM radius using the provided training and evaluation datasets.

    You should only consider radius values within the radius_to_consider list.
    You should use accuracy as a metric for comparing the different radius values.

    Args:
        train_matrix: The word counts for the training data
        train_labels: The spma or not spam labels for the training data
        val_matrix: The word counts for the validation data
        val_labels: The spam or not spam labels for the validation data
        radius_to_consider: The radius values to consider

    Returns:
        The best radius which maximizes SVM accuracy.
    """
    # *** START CODE HERE ***

    best_radius = -1
    best_acc = -1
    for radius in radius_to_consider:
        pred_svm = svm.train_and_predict_svm(train_matrix, train_labels, val_matrix, radius)
        accuracy = np.mean(pred_svm == val_labels)
        print(str(radius) + ',' + str(accuracy))
        if accuracy > best_acc:
            best_acc = accuracy
            best_radius = radius
    return radius

In [32]:
compute_best_svm_radius(transform_text(train_messages, word_dict), train_labels, 
                        transform_text(val_messages, word_dict), val_labels, [0.01, 0.1, 1, 10])

0.01,0.9156193895870736
0.1,0.9461400359066428
1,0.9281867145421903
10,0.8761220825852782


10